In [1]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


<span style="font-size:12px;">

FAISS
- FAISS에서 대규모 벡터 검색을 빠르고 효율적으로 하려고 만든 세 가지 핵심 접근 방식: IVF, PQ, HNSW

---
- <span style="color: Gold"> IVF (Inverted File Index) </span>
    - 개념: 벡터 공간을 여러 **클러스터(그룹)**로 나눔
    - 목적: 쿼리와 관련 없는 벡터는 탐색하지 않아 속도를 높임
    - 장점: 대규모 벡터에서 속도 빠름
    - 단점: 일부 정확도 손실 가능
    - 동작:
        - K-means로 클러스터(nlist) 생성
        - 각 벡터를 가장 가까운 클러스터에 할당
        - 검색 시 쿼리와 가까운 nprobe 클러스터만 탐색

- <span style="color: Gold"> PQ (Product Quantization)</span>
    - 개념: 고차원 벡터를 여러 서브벡터로 나눈 후, 각 서브벡터를 대표 값(클러스터)으로 근사
    - 목적: 메모리 절감 + 빠른 근사 거리 계산
    - 장점: 메모리 절감, 검색 속도 빠름
    - 단점: 근사로 인한 정확도 손실
    - 동작:
        - 벡터를 서브벡터로 분할 (예: 768차원 → 8개 × 96차원)
        - 각 서브벡터를 코드북에 가장 가까운 값으로 치환
        - 검색 시 코드만으로 근사 거리 계산

- <span style="color: Gold"> HNSW (Hierarchical Navigable Small World)</span>
    - 개념: 다층 그래프 구조에서 벡터를 연결해 근사 최근접 탐색
    - 목적: 빠른 검색 + 높은 정확도
    - 장점: 속도 매우 빠름, 정확도 높음
    - 단점: 메모리 중간 수준, 그래프 구축 필요
    - 동작:
        - 상위 레이어 → 장거리 점프 (빠른 후보 탐색)
        - 하위 레이어 → 지역 탐색 (정밀도)


In [2]:
# FAISS 사용
import faiss
import numpy as np

In [ ]:
# 데이터 생성
np.random.seed(42)
d= 128 # 벡터수
n = 10000 # 데이터 수
xb = np.random.randn(n,d).astype('float32') # 데이터베이스 / 랜덤 데이터베이스 생성
xq = np.random.randn(5,d).astype('float32') # 쿼리 / 랜덤 질문 쿼리 생성

# 정확한 검색
index_flat = faiss.IndexFlatL2(d) # 모든 DB 벡터와 L2 거리 계산
index_flat.add(xb)                # FAISS 인덱스에 벡터 추가

# 상위 4개 검색
k=4
D,I = index_flat.search(xq,k)
# k → 상위 k개 결과 검색
# D → 쿼리별 상위 k개 벡터와의 L2 거리
# I → 쿼리별 상위 k개 벡터 DB 인덱스 번호
# I[0] → 첫 번째 쿼리 벡터와 가장 가까운 DB 벡터 인덱스
print(f'Flat 인덱스 결과')
print(f'거리 : {D}')  # 쿼리와 DB 벡터 사이의 L2 거리 (작을수록 가까움)
print(f'인덱스 : {I}')


IFV 원리: 벡터 공간을 여러 클러스터로 분할하고, 검색 시 관련 클러스터만 탐색

- 나누는 이유 : ㅁ노든 벡터와 비교하면 느리기 때문에 클러스터 단위로 나누면 쿼리와 
관련된 벡터만 탐색 가능하여 속도가 대폭 개선된다 

2️⃣ 구조와 동작

학습 단계

K-means로 벡터 공간을 nlist 개의 클러스터로 나눔

각 클러스터 중심을 저장

벡터 추가 단계

각 벡터를 가장 가까운 클러스터에 할당

검색 단계

쿼리 벡터와 가장 가까운 nprobe 개 클러스터만 탐색

클러스터 내부는 Flat(L2) 또는 PQ로 거리 계산

=> 문장 벡터들 중에서 연관성 있는 문장 벡터끼리 클러스터로 묶어 관련있는 것만 검증

In [ ]:
d= 128 # 벡터수
n = 10000 # 데이터 수
xb = np.random.randn(n,d).astype('float32')# 데이터베이스
# IVF 인덱스 생성   # IVF (Inverted File Index): FAISS에서 사용하는 근사 검색(ANN) 최적화 기법. 벡터를 여러 **클러스터(그룹)**로 나누고, 검색할 때 일부 클러스터만 탐색하는 방식
# 클러스터 단위로 나누면 쿼리와 관련된 벡터만 탐색이 가능해 속도가 대폭 개선된다
nlist = 100 # 클러스터
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer,d,nlist)
# 학습(클러스터링)
index.train(xb)
# 데이터 추가
index.add(xb)
# 검색(nprobe 조절)
index.nprobe = 10 # 클러스터 수
D,I = index.search(xq, k=5)
print(f'IVF 인덱스 결과')
print(f'거리 : {D}') 
print(f'인덱스 : {I}')

- PQ (Product Quantization)
고차원 벡터를 여러 서브벡터로 나누고, 각 서브벡터를 양자화(quantize)하여 압축하는 기법

벡터는 원래 고차원인데 PQ에서는 이 벡터를 여러 작은 조작으로 나눈다. 차원이 낮은 단위로나누면 각 부분별로 근사화(압축)이 가능

- 서브벡터 : 원래 벡터가 768차원이라면 
서브벡터로 분할해서 이런식으로 

Subvector1 = [v1, ..., v96]
Subvector2 = [v97, ..., v192]
...
Subvector8 = [v673, ..., v768]

- 양자화 : 실제 벡터 값을 근사값으로 바꾸는 과정. 각 서브벡터를 가장 가까운 **코드북 중심(대표 값)**으로 바꿔 저장

서브벡터 분할 -> 서브벡터들 → 코드북학습(K-means) → centroid 집합 → 코드북 생성 -> 양자화


In [ ]:
d = 128 # 벡터수
n = 10000 # 데이터 수
xb = np.random.randn(n,d).astype('float32')  # 데이터베이스

# IVF + PQ 인덱스
nlist = 100 # 클러스터 수
m = 8 # 서브벡터 수( d가 m으로 나눠떨어져야 함)
nbits = 8  # 각 서브벡터당 비트 수( 2^8) = 256 클러스터

quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFPQ(quantizer,d,nlist,m,nbits)
# 학습
index.train(xb)
#추가
index.add(xb)
#검색
index.nprobe = 10
D,I =  index.search(xq, k=2)
# 메모리 비교
flat_memory = n*d*4
pq_memory = n*m  # 각 벡터당 m바이트
print(f'Flat 메모리 : {flat_memory / 1e6:.2f}MB')
print(f'PQ 메모리 : {pq_memory / 1e6:.2f}MB')
print(f'압축률 : {flat_memory / pq_memory:.0f}MB')


HNSW (Hierarchical Navigable Small World)
- FAISS에서 사용하는 근사 최근접 이웃 검색(ANN) 기법

다층 그래프 구조를 이용해 벡터 간 유사도를 빠르게 탐색

1️⃣ 구조

여러 **레이어(layer)**로 구성된 그래프

각 레이어에서 노드(벡터)끼리 연결되어 있음
상위 레이어 → 멀리 있는 후보 벡터로 빠르게 이동

하위 레이어 → 근처 벡터를 정밀 탐색

In [ ]:

d = 128 # 벡터수
n = 10000 # 데이터 수
xb = np.random.randn(n,d).astype('float32')  # 데이터베이스
# HNSW 인덱스
M=32 # 각 노드의 연결 수
index = faiss.IndexHNSWFlat(d,M)
# 인덱스 구축시 탐색 깊이
index.hnsw.efConstruction = 40
# 데이터 추가(학습불필요)
index.add(xb)
# 검색시 탐색 깊이
index.hnsw.efSearch = 16
D,I = index.search(xq, k=5)

import faiss
import numpy as np
import time

def benchmark_index(index, xb, xq, name):
    """인덱스 성능 벤치마크"""
    # 추가 시간
    start = time.time()
    if hasattr(index, 'train'):
        index.train(xb)
    index.add(xb)
    add_time = time.time() - start

    # 검색 시간
    start = time.time()
    for _ in range(10):
        index.search(xq, k=10)
    search_time = (time.time() - start) / 10

    print(f"{name}:")
    print(f"  추가 시간: {add_time:.3f}s")
    print(f"  검색 시간: {search_time*1000:.2f}ms")

# 데이터 준비
d, n = 128, 100000
xb = np.random.randn(n, d).astype('float32')
xq = np.random.randn(100, d).astype('float32')

# Flat
benchmark_index(faiss.IndexFlatL2(d), xb.copy(), xq, "Flat")

# IVF
nlist = 100
quantizer = faiss.IndexFlatL2(d)
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist)
benchmark_index(index_ivf, xb.copy(), xq, "IVF")

# HNSW
benchmark_index(faiss.IndexHNSWFlat(d, 32), xb.copy(), xq, "HNSW")